In [1]:
import transformers
import os
import math
print(transformers.__version__)
import torch
model_checkpoint = "bert-base-cased"
#model_checkpoint = "allenai/scibert_scivocab_cased"
#model_checkpoint = "roberta-large"
batch_size = 16
from transformers import AutoConfig,AutoModel
import transformers
from transformers import AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding,\
                                        TrainingArguments, Trainer,default_data_collator,AdamW
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,add_prefix_space=True)
from torch.utils.data import DataLoader,RandomSampler
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.optim.lr_scheduler import ExponentialLR
#tokenizer = RobertaTokenizerFast.from_pretrained(model_checkpoint,add_prefix_space=True)

4.12.3


In [2]:
from collections import Counter
fct = CrossEntropyLoss()

In [3]:
from datasets import load_dataset,load_metric
import datasets
import numpy as np
import random

In [4]:
f1_metric = load_metric("f1")
pr_metric = load_metric('precision')
re_metric = load_metric('recall')

In [5]:
from transformers.models.bert.modeling_bert import BertPooler

In [22]:
class PretrainedSequenceModel(torch.nn.Module):
    def __init__(self,labels):
        super().__init__()
        self.num_labels = labels
        self.base_model = AutoModel.from_pretrained(model_checkpoint,output_hidden_states=False,add_pooling_layer=False)
        #Pretrained Using MLM and saved 
        #self.base_model.load_state_dict(torch.load("mlm_electronics/electronics_amazon.bin"))
        self.dropout = torch.nn.Dropout(self.base_model.config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(self.base_model.config.hidden_size,self.num_labels)
        self._init_weights(self.classifier)
        self.pooler = BertPooler(self.base_model.config)
        self._init_weights(self.pooler)
        
    def _init_weights(self, modules):
        """Initialize the weights"""
        for module in modules.modules():
                if isinstance(module, torch.nn.Linear):
                    module.weight.data.normal_(mean=0.0, std=self.base_model.config.initializer_range)
                    if module.bias is not None:
                        module.bias.data.zero_()
                elif isinstance(module, torch.nn.LayerNorm):
                    module.bias.data.zero_()
                    module.weight.data.fill_(1.0)
        

    def forward(self,data):
        out = self.base_model(input_ids=data['input_ids'], \
                               attention_mask=data['attention_mask'])
        out = self.pooler(out.last_hidden_state)
        clf_out = self.classifier(self.dropout(out))
        return clf_out


In [7]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def preprocess_function(examples):
        result = tokenizer(examples['text'], padding="max_length", max_length=512, truncation=True)
        return result

def compute_metrics(p):
    f1_metric = load_metric("f1")
    pr_metric = load_metric('precision')
    re_metric = load_metric('recall')
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)
    results1 = f1_metric.compute(predictions=predictions, references=labels,average="weighted")
    results2 = pr_metric.compute(predictions=predictions, references=labels,average="weighted")
    results3 = re_metric.compute(predictions=predictions, references=labels,average="weighted")
    return {
        "precision": results2["precision"],
        "recall": results3["recall"],
        "f1": results1["f1"],
    }

In [8]:
np.random.randint(0,100000)

88717

In [9]:
def baseline1():   
            
            seed = np.random.randint(0,1000000)
            fix_all_seeds(seed)
            dataset = load_dataset('csv',delimiter="\t",data_files='books/review_labels.csv')
            dataset = datasets.concatenate_datasets([dataset['train']])
            dataset_src = dataset.train_test_split(0.2,shuffle=False)
            
            dataset = load_dataset('csv',delimiter="\t",data_files='electronics/review_labels.csv')
            dataset = datasets.concatenate_datasets([dataset['train']])
            dataset_trg = dataset.train_test_split(0.2,shuffle=False)
            
            processed_datasets_src = dataset_src.map(preprocess_function,batched=True,\
                                      desc="Running tokenizer on dataset",)

            processed_datasets_trg = dataset_trg.map(preprocess_function,batched=True,\
                                      desc="Running tokenizer on dataset",)
            
            processed_datasets_src.remove_columns_(["text"])
            processed_datasets_trg.remove_columns_(["text"])
            
            config = AutoConfig.from_pretrained(model_checkpoint,num_labels=2,)
            model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,config=config,)
            
            args = TrainingArguments(
                "sanity-chunk",
                evaluation_strategy = "epoch",
                learning_rate=5e-5,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                num_train_epochs=5,
                weight_decay=0.01,
                save_strategy="epoch",
                logging_steps=100,
                overwrite_output_dir=True,
                load_best_model_at_end=True,
                metric_for_best_model = "eval_f1",
                seed = seed,
            )
            
            trainer = Trainer(
                    model,
                    args,
                    train_dataset=processed_datasets_src['train'],
                    eval_dataset= processed_datasets_src['test'],
                    data_collator=default_data_collator,
                    tokenizer=tokenizer,
                    compute_metrics=compute_metrics
                )
            
            trainer.train(resume_from_checkpoint=None)
            #p = trainer.predict(processed_datasets_src['test'])
            p = trainer.predict(processed_datasets_trg['test'])
            
            y_hat = np.argmax(p.predictions,1)
            y = p.label_ids
            out = f1_metric.compute(predictions=y_hat,references=y)
            return out['f1']

In [23]:
def run_train(final_train_loader,final_eval_loader):
            model = PretrainedSequenceModel(2)
            model.cuda()
            no_decay = ["bias", "LayerNorm.weight"]
            optimizer_grouped_parameters = [
                    {
                            "params": [p for n, p in model.named_parameters() \
                                       if not any(nd in n for nd in no_decay)],
                            "weight_decay": 1e-2,
                    },
                    {
                        "params": [p for n, p in model.named_parameters() \
                                   if any(nd in n for nd in no_decay)],
                        "weight_decay": 0.0,
                    },
                        ]
            optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)
            fct_loss = CrossEntropyLoss()
            scheduler = ExponentialLR(optimizer=optimizer,gamma=0.85,last_epoch=-1,verbose=True)
            
            best_f1 = -1
            for epoch in range(10):
                print(f'EPOCH NO: {epoch}')
                model.eval()
                val_loss = 0.0
                token_predictions_store = []
                token_gold_store = []
                for step, batch in enumerate(final_eval_loader):
                    with torch.no_grad():
                        data = {'input_ids':batch['input_ids'].cuda(),\
                               'attention_mask':batch['attention_mask'].cuda(),\
                               'labels':batch['labels'].cuda()}
                        out = model(data)
                        token_predictions_store.append(out)
                        token_gold_store.append(batch['labels'])
                        loss = fct(out.view(-1,model.num_labels),\
                                                   data['labels'].view(-1))
                        val_loss = val_loss + loss.item()

                predictions = torch.vstack(token_predictions_store)
                references = torch.hstack(token_gold_store)
                predictions = torch.argmax(predictions,dim=-1)
                print(predictions.shape,references.shape)
                y_pred = predictions.detach().cpu().clone().numpy()
                y_true = references.detach().cpu().clone().numpy()
                print(y_pred.shape,y_true.shape)
                eval_f1 = f1_metric.compute(predictions=y_pred, references=y_true)
                print('-'*100)
                print(eval_f1)
                print(f'Epoch {epoch} validation loss {val_loss/len(final_eval_loader)}')
                if eval_f1['f1'] > best_f1:
                    best_f1 = eval_f1['f1']
                    torch.save(model.state_dict(),"saved_model/pretrained_amazon.bin")
                print('-'*100)
        
                model.train()
                epoch_loss = 0.0
                for step, batch in enumerate(final_train_loader):
                    data = {'input_ids':batch['input_ids'].cuda(),\
                               'attention_mask':batch['attention_mask'].cuda(),\
                               'labels':batch['labels'].cuda()}
                    optimizer.zero_grad()
                    out = model(data)
                    loss = fct(out.view(-1,model.num_labels),\
                                               data['labels'].view(-1))
                    epoch_loss = epoch_loss + loss.item()
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                print(f'Epoch {epoch} training loss {epoch_loss/len(final_train_loader)}')
                print('**************************************************************************')
            print(f'Best F1 score{best_f1}')

In [24]:
def run_test(data):
    
        model = PretrainedSequenceModel(2)
        model.cuda()
        model.load_state_dict(torch.load("saved_model/pretrained_amazon.bin"))
        model.eval()
        token_predictions_store = []
        token_gold_store = []
        for step, batch in enumerate(data):
                    with torch.no_grad():
                        data = {'input_ids':batch['input_ids'].cuda(),\
                               'attention_mask':batch['attention_mask'].cuda(),\
                               'labels':batch['labels'].cuda()}
                        out = model(data)
                        token_predictions_store.append(out)
                        token_gold_store.append(data['labels'])
                        loss = fct(out.view(-1,model.num_labels),\
                                                   data['labels'].view(-1))
                        loss = loss + loss.item()

        predictions = torch.vstack(token_predictions_store)
        references = torch.hstack(token_gold_store)
        predictions = torch.argmax(predictions,dim=-1)
        print(predictions.shape,references.shape)
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()
        test_f1 = f1_metric.compute(predictions=y_pred, references=y_true)
        print(f'Test F1 score {test_f1}')
        return test_f1['f1']

In [25]:
def baseline2():
        seed = np.random.randint(0,1000000)
        fix_all_seeds(seed)
        dataset = load_dataset('csv',delimiter="\t",data_files='books/review_labels.csv')
        dataset = datasets.concatenate_datasets([dataset['train']])
        dataset_src = dataset.train_test_split(0.2,shuffle=False)

        dataset = load_dataset('csv',delimiter="\t",data_files='electronics/review_labels.csv')
        dataset = datasets.concatenate_datasets([dataset['train']])
        dataset_trg = dataset.train_test_split(0.2,shuffle=False)

        processed_datasets_src = dataset_src.map(preprocess_function,batched=True,\
                                  desc="Running tokenizer on dataset",)

        processed_datasets_trg = dataset_trg.map(preprocess_function,batched=True,\
                                  desc="Running tokenizer on dataset",)

        processed_datasets_src.remove_columns_(["text"])
        processed_datasets_trg.remove_columns_(["text"])


        train_dataloader_src =DataLoader(processed_datasets_src['train'],\
                                                     collate_fn=default_data_collator,\
                                                     batch_size = 16,drop_last=True)
        eval_dataloader_src = DataLoader(processed_datasets_src['test'],\
                                         collate_fn=default_data_collator,\
                                         batch_size = 16,drop_last=True)
        test_dataloader_tgt = DataLoader(processed_datasets_trg['test'],\
                                         collate_fn=default_data_collator,\
                                         batch_size = 16,drop_last=True)
        run_train(train_dataloader_src,eval_dataloader_src)
        return run_test(test_dataloader_tgt)

In [26]:
output = []
for i in range(5):
            output.append(baseline2())

print(np.mean(output),np.std(output))    

Using custom data configuration default-34360c82fb13fe72
Reusing dataset csv (/ukp-storage-1/sarkar/.cache/huggingface/datasets/csv/default-34360c82fb13fe72/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-6ce991164e62b0c8
Reusing dataset csv (/ukp-storage-1/sarkar/.cache/huggingface/datasets/csv/default-6ce991164e62b0c8/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Running tokenizer on dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

...

Running tokenizer on dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.bias', 'cls.predictions.bias', 'bert.pooler.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Adjusting learning rate of group 0 to 2.0000e-05.
Adjusting learning rate of group 1 to 2.0000e-05.
EPOCH NO: 0
torch.Size([400]) torch.Size([400])
(400,) (400,)
----------------------------------------------------------------------------------------------------
{'f1': 0.6298932384341637}
Epoch 0 validation loss 0.6988060665130615
----------------------------------------------------------------------------------------------------
Adjusting learning rate of group 0 to 1.7000e-05.
Adjusting learning rate of group 1 to 1.7000e-05.
Epoch 0 training loss 0.4637314909696579
**************************************************************************
EPOCH NO: 1
torch.Size([400]) torch.Size([400])
(400,) (400,)
----------------------------------------------------------------------------------------------------
{'f1': 0.8797953964194373}
Epoch 1 validation loss 0.33085944324731825
----------------------------------------------------------------------------------------------------
Adjusting learn

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['bert.pooler.dense.bias', 'cls.predictions.bias', 'bert.pooler.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([400]) torch.Size([400])
Test F1 score {'f1': 0.7899686520376177}
0.808884076446256 0.033393342865731174


In [27]:
((0.926-0.82786)/0.926)*100

10.59827213822894